In [1]:
# system packages
import sys
import time
import warnings
import os

# non-geo numeric packages
import numpy as np
import math
from itertools import product, combinations
import pandas as pd

# network and OSM packages
import networkx as nx
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

# Earth engine packages
import ee
import geemap

# General geo-packages
import libpysal
import rasterio
import geopandas as gpd
import shapely
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon, MultiPolygon

In [2]:
# Authenticate and Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VMXtVcwE5YXn_QakrHFcs3AFpgeeIS_515yUZLytQJ27Lt9NFwnoR0

Successfully saved authorization token.


In [14]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Hue'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Hue', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Vietnam']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\VNM_Hue_2020.tif
get road networks from OSM
Hue done 0.58 mns
 
get urban greenspaces from OSM
Hue done
 
100m resolution grids extraction
Hue 1.18 mns

Hue
0.0 % fake entry points done 0.0  mns
99.0 % fake entry points done 0.2  mns
Hue 100 % fake entry points done 0.2  mns
Hue 100 % done 0.21  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
100 % finding combinations done
Hue 21586 suitible combinations

obtain local graphs
Hue
0.0 % done 0.06 mns
100 % done 0.09 mns

Hue
0.0 % 0.07 mns
100 % done 0.57 mns

Hue
entrance 0.01 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 0.51 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 1.01 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 1.54 mns
grid  300
grid  600
grid  1000
Hue done 2.04 mns
CPU times: total: 4min 21s
Wall time: 5min 58s


City                               Hue
entrance_300        1 high    0.068910
                    2 medium  0.009432
                    3 low     0.047558
                    4 no      0.874100
entrance_600        1 high    0.089143
                    2 medium  0.061669
                    3 low     0.139155
                    4 no      0.710033
entrance_1000       1 high    0.137927
                    2 medium  0.123504
                    3 low     0.092474
                    4 no      0.646094
gravity**(1/2)_300  1 high    0.068793
                    2 medium  0.004914
                    3 low     0.032929
                    4 no      0.893364
gravity**(1/2)_600  1 high    0.074248
                    2 medium  0.040811
                    3 low     0.134701
                    4 no      0.750240
gravity**(1/2)_1000 1 high    0.102941
                    2 medium  0.114903
                    3 low     0.109262
                    4 no      0.672894
gravity**(1/3)_300  1 high    0.068910
                    2 medium  0.004936
                    3 low     0.032412
                    4 no      0.893742
gravity**(1/3)_600  1 high    0.074775
                    2 medium  0.042317
                    3 low     0.139188
                    4 no      0.743720
gravity**(1/3)_1000 1 high    0.107018
                    2 medium  0.119522
                    3 low     0.115725
                    4 no      0.657735
gravity**(1/5)_300  1 high    0.068910
                    2 medium  0.004701
                    3 low     0.039532
                    4 no      0.886857
gravity**(1/5)_600  1 high    0.078704
                    2 medium  0.046400
                    3 low     0.139301
                    4 no      0.735595
gravity**(1/5)_1000 1 high    0.120592
                    2 medium  0.119527
                    3 low     0.112290
                    4 no      0.647591

In [13]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Manila'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Manila', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Philippines']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\PHL_Manila_2020.tif
get road networks from OSM
Manila done 2.58 mns
 
get urban greenspaces from OSM
Manila done
 
100m resolution grids extraction
Manila 3.44 mns

Manila
0.0 % fake entry points done 0.01  mns
8.8 % fake entry points done 1.14  mns
17.6 % fake entry points done 2.16  mns
26.4 % fake entry points done 3.48  mns
35.2 % fake entry points done 4.58  mns
44.0 % fake entry points done 5.62  mns
52.8 % fake entry points done 6.93  mns
61.6 % fake entry points done 8.13  mns
70.4 % fake entry points done 9.18  mns
79.2 % fake entry points done 10.41  mns
88.0 % fake entry points done 11.56  mns
96.8 % fake entry points done 12.72  mns
Manila 100 % fake entry points done 13.08  mns
Manila 100 % done 13.12  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
13.09 % 0.46 mns
26.18 % 0.96 mns
39.27 % 1.5 mns
52.36 % 2.01 mns
65.45 % 2.55 mns
78.53 % 3.1 mns
91.62 % 3.7 mns
100 % fi

City                            Manila
entrance_300        1 high    0.061143
                    2 medium  0.015303
                    3 low     0.068714
                    4 no      0.854840
entrance_600        1 high    0.082389
                    2 medium  0.085341
                    3 low     0.206535
                    4 no      0.625735
entrance_1000       1 high    0.173767
                    2 medium  0.181694
                    3 low     0.223927
                    4 no      0.420613
gravity**(1/2)_300  1 high    0.062273
                    2 medium  0.029740
                    3 low     0.098046
                    4 no      0.809941
gravity**(1/2)_600  1 high    0.094104
                    2 medium  0.130803
                    3 low     0.280536
                    4 no      0.494557
gravity**(1/2)_1000 1 high    0.231705
                    2 medium  0.247503
                    3 low     0.237951
                    4 no      0.282841
gravity**(1/3)_300  1 high    0.061171
                    2 medium  0.017830
                    3 low     0.069553
                    4 no      0.851446
gravity**(1/3)_600  1 high    0.079317
                    2 medium  0.088599
                    3 low     0.221912
                    4 no      0.610172
gravity**(1/3)_1000 1 high    0.174687
                    2 medium  0.209880
                    3 low     0.283763
                    4 no      0.331670
gravity**(1/5)_300  1 high    0.060791
                    2 medium  0.014040
                    3 low     0.060503
                    4 no      0.864665
gravity**(1/5)_600  1 high    0.075476
                    2 medium  0.077416
                    3 low     0.198032
                    4 no      0.649076
gravity**(1/5)_1000 1 high    0.156294
                    2 medium  0.189612
                    3 low     0.269065
                    4 no      0.385029

In [12]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Yaounde'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Yaounde', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Cameroon']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\CMR_Yaounde_2020.tif
get road networks from OSM
Yaounde done 0.78 mns
 
get urban greenspaces from OSM
Yaounde done
 
100m resolution grids extraction
Yaounde 1.78 mns

Yaounde
0.0 % fake entry points done 0.01  mns
44.6 % fake entry points done 0.42  mns
89.3 % fake entry points done 0.73  mns
Yaounde 100 % fake entry points done 0.81  mns
Yaounde 100 % done 0.84  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
36.31 % 0.21 mns
72.62 % 0.48 mns
100 % finding combinations done
Yaounde 977738 suitible combinations

obtain local graphs
Yaounde
0.0 % done 0.38 mns
36.31 % done 0.42 mns
72.62 % done 0.5 mns
100 % done 0.59 mns

Yaounde
0.0 % 0.14 mns
4.44 % 0.75 mns
8.88 % 1.36 mns
13.32 % 1.95 mns
17.76 % 2.58 mns
22.2 % 3.18 mns
26.64 % 3.8 mns
31.08 % 4.41 mns
35.52 % 5.11 mns
39.96 % 5.7 mns
44.4 % 6.34 mns
48.84 % 6.97 mns
53.28 % 7.56 mns
57.71 % 8.19 mns
62.15 % 8.87 mns
66.59 % 9.5 m

City                           Yaounde
entrance_300        1 high    0.081629
                    2 medium  0.018829
                    3 low     0.050997
                    4 no      0.848545
entrance_600        1 high    0.101755
                    2 medium  0.063767
                    3 low     0.141215
                    4 no      0.693262
entrance_1000       1 high    0.167898
                    2 medium  0.134520
                    3 low     0.251949
                    4 no      0.445633
gravity**(1/2)_300  1 high    0.081148
                    2 medium  0.014383
                    3 low     0.034067
                    4 no      0.870402
gravity**(1/2)_600  1 high    0.090929
                    2 medium  0.045886
                    3 low     0.129795
                    4 no      0.733389
gravity**(1/2)_1000 1 high    0.134896
                    2 medium  0.122879
                    3 low     0.228816
                    4 no      0.513409
gravity**(1/3)_300  1 high    0.080977
                    2 medium  0.011897
                    3 low     0.036313
                    4 no      0.870813
gravity**(1/3)_600  1 high    0.090331
                    2 medium  0.045076
                    3 low     0.107419
                    4 no      0.757174
gravity**(1/3)_1000 1 high    0.127315
                    2 medium  0.103749
                    3 low     0.216547
                    4 no      0.552390
gravity**(1/5)_300  1 high    0.081077
                    2 medium  0.012717
                    3 low     0.037924
                    4 no      0.868282
gravity**(1/5)_600  1 high    0.090908
                    2 medium  0.048916
                    3 low     0.108772
                    4 no      0.751404
gravity**(1/5)_1000 1 high    0.132331
                    2 medium  0.109917
                    3 low     0.219289
                    4 no      0.538462

In [13]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Lubumbashi'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Lubumbashi', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Democratic Republic of the Congo']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\COD_Lubumbashi_2020.tif
get road networks from OSM
Lubumbashi done 0.72 mns
 
get urban greenspaces from OSM
Lubumbashi done
 
100m resolution grids extraction
Lubumbashi 0.78 mns

Lubumbashi
0.0 % fake entry points done 0.0  mns
Lubumbashi 100 % fake entry points done 0.03  mns
Lubumbashi 100 % done 0.04  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
100 % finding combinations done
Lubumbashi 27882 suitible combinations

obtain local graphs
Lubumbashi
0.0 % done 0.02 mns
100 % done 0.02 mns

Lubumbashi
0.0 % 0.06 mns
100 % done 0.56 mns

Lubumbashi
entrance 0.01 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 1.54 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 3.12 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 4.64 mns
grid  300
grid  600
grid  1000
Lubumbashi done 6.15 mns
CPU times: total: 7min 52s
Wall time: 8min 31s


City                          Lubumbashi
entrance_300        1 high      0.002246
                    2 medium    0.001321
                    3 low       0.004134
                    4 no        0.992299
entrance_600        1 high      0.002246
                    2 medium    0.005621
                    3 low       0.018374
                    4 no        0.973759
entrance_1000       1 high      0.002727
                    2 medium    0.016577
                    3 low       0.050250
                    4 no        0.930446
gravity**(1/2)_300  1 high      0.002246
                    2 medium    0.000747
                    3 low       0.003159
                    4 no        0.993848
gravity**(1/2)_600  1 high      0.002246
                    2 medium    0.003906
                    3 low       0.010602
                    4 no        0.983246
gravity**(1/2)_1000 1 high      0.002368
                    2 medium    0.009819
                    3 low       0.031817
                    4 no        0.955997
gravity**(1/3)_300  1 high      0.002246
                    2 medium    0.000865
                    3 low       0.003252
                    4 no        0.993637
gravity**(1/3)_600  1 high      0.002246
                    2 medium    0.004166
                    3 low       0.010270
                    4 no        0.983318
gravity**(1/3)_1000 1 high      0.002392
                    2 medium    0.009918
                    3 low       0.033266
                    4 no        0.954424
gravity**(1/5)_300  1 high      0.002246
                    2 medium    0.000936
                    3 low       0.003393
                    4 no        0.993425
gravity**(1/5)_600  1 high      0.002246
                    2 medium    0.004378
                    3 low       0.011321
                    4 no        0.982055
gravity**(1/5)_1000 1 high      0.002443
                    2 medium    0.010654
                    3 low       0.038445
                    4 no        0.948459

In [14]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Guatemala City'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Guatemala City', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Guatemala']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\GTM_Guatemala City_2020.tif
get road networks from OSM
Guatemala City done 1.27 mns
 
get urban greenspaces from OSM
Guatemala City done
 
100m resolution grids extraction
Guatemala City 1.61 mns

Guatemala City
0.0 % fake entry points done 0.01  mns
11.5 % fake entry points done 0.38  mns
23.0 % fake entry points done 0.77  mns
34.5 % fake entry points done 1.17  mns
46.0 % fake entry points done 1.57  mns
57.5 % fake entry points done 1.98  mns
69.0 % fake entry points done 2.4  mns
80.6 % fake entry points done 2.78  mns
92.1 % fake entry points done 3.16  mns
Guatemala City 100 % fake entry points done 3.42  mns
Guatemala City 100 % done 3.44  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
31.85 % 0.26 mns
63.69 % 0.54 mns
95.54 % 0.83 mns
100 % finding combinations done
Guatemala City 86042 suitible combinations

obtain local graphs
Guatemala City
0.0 % done 0.73 mns
31.85 % done 

City                          Guatemala City
entrance_300        1 high          0.115183
                    2 medium        0.003307
                    3 low           0.026636
                    4 no            0.854874
entrance_600        1 high          0.126035
                    2 medium        0.031162
                    3 low           0.070522
                    4 no            0.772281
entrance_1000       1 high          0.149481
                    2 medium        0.063320
                    3 low           0.068564
                    4 no            0.718635
gravity**(1/2)_300  1 high          0.114965
                    2 medium        0.008013
                    3 low           0.044198
                    4 no            0.832825
gravity**(1/2)_600  1 high          0.127667
                    2 medium        0.051724
                    3 low           0.080023
                    4 no            0.740585
gravity**(1/2)_1000 1 high          0.152518
                    2 medium        0.094683
                    3 low           0.045974
                    4 no            0.706825
gravity**(1/3)_300  1 high          0.114848
                    2 medium        0.004513
                    3 low           0.029738
                    4 no            0.850901
gravity**(1/3)_600  1 high          0.123450
                    2 medium        0.036962
                    3 low           0.087841
                    4 no            0.751747
gravity**(1/3)_1000 1 high          0.149687
                    2 medium        0.080612
                    3 low           0.062291
                    4 no            0.707410
gravity**(1/5)_300  1 high          0.114994
                    2 medium        0.003096
                    3 low           0.025052
                    4 no            0.856858
gravity**(1/5)_600  1 high          0.122476
                    2 medium        0.031383
                    3 low           0.081606
                    4 no            0.764534
gravity**(1/5)_1000 1 high          0.148241
                    2 medium        0.070526
                    3 low           0.073400
                    4 no            0.707834

In [15]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Perth'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Perth', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Australia']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\AUS_Perth_2020.tif
get road networks from OSM
Perth done 5.59 mns
 
get urban greenspaces from OSM
Perth done
 
100m resolution grids extraction
Perth 13.41 mns

Perth
0.0 % fake entry points done 0.01  mns
5.0 % fake entry points done 1.66  mns
9.9 % fake entry points done 2.81  mns
14.9 % fake entry points done 3.9  mns
19.8 % fake entry points done 4.99  mns
24.8 % fake entry points done 6.12  mns
29.7 % fake entry points done 7.15  mns
34.7 % fake entry points done 8.25  mns
39.7 % fake entry points done 9.31  mns
44.6 % fake entry points done 10.4  mns
49.6 % fake entry points done 11.42  mns
54.5 % fake entry points done 12.58  mns
59.5 % fake entry points done 13.67  mns
64.5 % fake entry points done 14.72  mns
69.4 % fake entry points done 15.79  mns
74.4 % fake entry points done 16.82  mns
79.3 % fake entry points done 17.85  mns
84.3 % fake entry points done 18.86  mns
89.2 % fake entr

City                             Perth
entrance_300        1 high    0.198796
                    2 medium  0.080576
                    3 low     0.218964
                    4 no      0.501665
entrance_600        1 high    0.344925
                    2 medium  0.259205
                    3 low     0.265972
                    4 no      0.129898
entrance_1000       1 high    0.743694
                    2 medium  0.164111
                    3 low     0.071939
                    4 no      0.020255
gravity**(1/2)_300  1 high    0.196868
                    2 medium  0.082626
                    3 low     0.204015
                    4 no      0.516491
gravity**(1/2)_600  1 high    0.288695
                    2 medium  0.280340
                    3 low     0.285705
                    4 no      0.145260
gravity**(1/2)_1000 1 high    0.662720
                    2 medium  0.227100
                    3 low     0.084266
                    4 no      0.025914
gravity**(1/3)_300  1 high    0.195696
                    2 medium  0.070051
                    3 low     0.193927
                    4 no      0.540326
gravity**(1/3)_600  1 high    0.276744
                    2 medium  0.259585
                    3 low     0.292827
                    4 no      0.170844
gravity**(1/3)_1000 1 high    0.634975
                    2 medium  0.235146
                    3 low     0.103480
                    4 no      0.026399
gravity**(1/5)_300  1 high    0.195602
                    2 medium  0.067233
                    3 low     0.196396
                    4 no      0.540769
gravity**(1/5)_600  1 high    0.283688
                    2 medium  0.257572
                    3 low     0.290448
                    4 no      0.168292
gravity**(1/5)_1000 1 high    0.652165
                    2 medium  0.221270
                    3 low     0.101987
                    4 no      0.024578

In [12]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['New Delhi'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_New Delhi', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['India']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\IND_New Delhi_2020.tif
get road networks from OSM
New Delhi done 3.61 mns
 
get urban greenspaces from OSM
New Delhi done
 
100m resolution grids extraction
New Delhi 9.68 mns

New Delhi
0.0 % fake entry points done 0.03  mns
3.9 % fake entry points done 1.57  mns
7.8 % fake entry points done 2.99  mns
11.6 % fake entry points done 4.4  mns
15.5 % fake entry points done 5.96  mns
19.4 % fake entry points done 7.49  mns
23.3 % fake entry points done 8.89  mns
27.2 % fake entry points done 10.4  mns
31.0 % fake entry points done 11.98  mns
34.9 % fake entry points done 13.5  mns
38.8 % fake entry points done 14.98  mns
42.7 % fake entry points done 16.57  mns
46.5 % fake entry points done 18.17  mns
50.4 % fake entry points done 19.89  mns
54.3 % fake entry points done 21.38  mns
58.2 % fake entry points done 22.88  mns
62.1 % fake entry points done 24.46  mns
65.9 % fake entry points done 25.91  mns


City                          New Delhi
entrance_300        1 high     0.168252
                    2 medium   0.036633
                    3 low      0.087089
                    4 no       0.708026
entrance_600        1 high     0.258223
                    2 medium   0.082232
                    3 low      0.132346
                    4 no       0.527199
entrance_1000       1 high     0.397483
                    2 medium   0.092584
                    3 low      0.110999
                    4 no       0.398934
gravity**(1/2)_300  1 high     0.174020
                    2 medium   0.068901
                    3 low      0.139063
                    4 no       0.618016
gravity**(1/2)_600  1 high     0.320807
                    2 medium   0.144818
                    3 low      0.175565
                    4 no       0.358810
gravity**(1/2)_1000 1 high     0.534867
                    2 medium   0.113521
                    3 low      0.095179
                    4 no       0.256433
gravity**(1/3)_300  1 high     0.166693
                    2 medium   0.048226
                    3 low      0.108284
                    4 no       0.676797
gravity**(1/3)_600  1 high     0.264140
                    2 medium   0.116373
                    3 low      0.164687
                    4 no       0.454800
gravity**(1/3)_1000 1 high     0.455955
                    2 medium   0.117138
                    3 low      0.130632
                    4 no       0.296276
gravity**(1/5)_300  1 high     0.164893
                    2 medium   0.039975
                    3 low      0.095736
                    4 no       0.699396
gravity**(1/5)_600  1 high     0.250245
                    2 medium   0.099903
                    3 low      0.148636
                    4 no       0.501215
gravity**(1/5)_1000 1 high     0.413625
                    2 medium   0.107887
                    3 low      0.129194
                    4 no       0.349293

In [14]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Shiraz'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Shiraz', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Iran']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\IRN_Shiraz_2020.tif
get road networks from OSM
Shiraz done 0.79 mns
 
get urban greenspaces from OSM
Shiraz done
 
100m resolution grids extraction
Shiraz 1.48 mns

Shiraz
0.0 % fake entry points done 0.0  mns
27.1 % fake entry points done 0.43  mns
54.2 % fake entry points done 0.88  mns
81.3 % fake entry points done 1.3  mns
Shiraz 100 % fake entry points done 1.63  mns
Shiraz 100 % done 1.65  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
20.11 % 0.34 mns
40.23 % 0.74 mns
60.34 % 1.16 mns
80.45 % 1.6 mns
100 % finding combinations done
Shiraz 1509131 suitible combinations

obtain local graphs
Shiraz
0.0 % done 1.23 mns
20.11 % done 1.33 mns
40.23 % done 1.45 mns
60.34 % done 1.62 mns
80.45 % done 1.84 mns
100 % done 2.11 mns

Shiraz
0.0 % 0.1 mns
2.42 % 0.7 mns
4.83 % 1.46 mns
7.25 % 2.03 mns
9.66 % 2.64 mns
12.08 % 3.29 mns
14.49 % 3.88 mns
16.91 % 4.56 mns
19.32 % 5.17 mns
21.74 % 5.76

City                            Shiraz
entrance_300        1 high    0.156136
                    2 medium  0.035591
                    3 low     0.102497
                    4 no      0.705775
entrance_600        1 high    0.197722
                    2 medium  0.133897
                    3 low     0.234313
                    4 no      0.434068
entrance_1000       1 high    0.383938
                    2 medium  0.207287
                    3 low     0.201518
                    4 no      0.207258
gravity**(1/2)_300  1 high    0.157394
                    2 medium  0.048444
                    3 low     0.115635
                    4 no      0.678528
gravity**(1/2)_600  1 high    0.207204
                    2 medium  0.150625
                    3 low     0.237957
                    4 no      0.404214
gravity**(1/2)_1000 1 high    0.402675
                    2 medium  0.210640
                    3 low     0.196367
                    4 no      0.190319
gravity**(1/3)_300  1 high    0.156229
                    2 medium  0.035461
                    3 low     0.097343
                    4 no      0.710968
gravity**(1/3)_600  1 high    0.185627
                    2 medium  0.131036
                    3 low     0.226465
                    4 no      0.456872
gravity**(1/3)_1000 1 high    0.336581
                    2 medium  0.220270
                    3 low     0.230055
                    4 no      0.213094
gravity**(1/5)_300  1 high    0.155769
                    2 medium  0.031246
                    3 low     0.092925
                    4 no      0.720060
gravity**(1/5)_600  1 high    0.183763
                    2 medium  0.122513
                    3 low     0.224289
                    4 no      0.469434
gravity**(1/5)_1000 1 high    0.326655
                    2 medium  0.217730
                    3 low     0.234095
                    4 no      0.221520

In [23]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Curitiba'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Curitiba', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

Warsaw
0.0 % 0.23 mns
1.03 % 0.85 mns
2.06 % 1.45 mns
3.08 % 2.03 mns
4.11 % 2.75 mns
5.14 % 3.4 mns
6.17 % 3.97 mns
7.2 % 4.53 mns
8.23 % 5.07 mns
9.25 % 5.63 mns
10.28 % 6.18 mns
11.31 % 6.75 mns
12.34 % 7.35 mns
13.37 % 7.93 mns
14.39 % 8.53 mns
15.42 % 9.13 mns
16.45 % 9.7 mns
17.48 % 10.27 mns
18.51 % 10.86 mns
19.54 % 11.48 mns
20.56 % 12.04 mns
21.59 % 12.66 mns
22.62 % 13.21 mns
23.65 % 13.78 mns
24.68 % 14.45 mns
25.7 % 14.99 mns
26.73 % 15.55 mns
27.76 % 16.12 mns
28.79 % 16.81 mns
29.82 % 17.46 mns
30.85 % 18.1 mns
31.87 % 18.73 mns
32.9 % 19.48 mns
33.93 % 20.18 mns
34.96 % 20.76 mns
35.99 % 21.52 mns
37.01 % 22.17 mns
38.04 % 22.73 mns
39.07 % 23.35 mns
40.1 % 23.92 mns
41.13 % 24.52 mns
42.16 % 25.15 mns
43.18 % 25.74 mns
44.21 % 26.35 mns
45.24 % 26.96 mns
46.27 % 27.6 mns
47.3 % 28.29 mns
48.32 % 28.9 mns
49.35 % 29.56 mns
50.38 % 30.36 mns
51.41 % 31.12 mns
52.44 % 31.72 mns
53.47 % 32.51 mns
54.49 % 33.3 mns
55.52 % 33.97 mns
56.55 % 34.67 mns
57.58 % 35.41 mns
58.61 

City                            Warsaw
entrance_300        1 high    0.118107
                    2 medium  0.066353
                    3 low     0.191894
                    4 no      0.623646
entrance_600        1 high    0.261987
                    2 medium  0.216992
                    3 low     0.279017
                    4 no      0.242004
entrance_1000       1 high    0.612042
                    2 medium  0.179275
                    3 low     0.140955
                    4 no      0.067729
gravity**(1/2)_300  1 high    0.127733
                    2 medium  0.098925
                    3 low     0.221482
                    4 no      0.551861
gravity**(1/2)_600  1 high    0.311592
                    2 medium  0.258176
                    3 low     0.266758
                    4 no      0.163473
gravity**(1/2)_1000 1 high    0.730296
                    2 medium  0.149326
                    3 low     0.104441
                    4 no      0.015937
gravity**(1/3)_300  1 high    0.116542
                    2 medium  0.072202
                    3 low     0.175256
                    4 no      0.636000
gravity**(1/3)_600  1 high    0.234220
                    2 medium  0.212200
                    3 low     0.294395
                    4 no      0.259185
gravity**(1/3)_1000 1 high    0.565233
                    2 medium  0.226364
                    3 low     0.150926
                    4 no      0.057477
gravity**(1/5)_300  1 high    0.114354
                    2 medium  0.061485
                    3 low     0.170626
                    4 no      0.653536
gravity**(1/5)_600  1 high    0.218446
                    2 medium  0.209660
                    3 low     0.286410
                    4 no      0.285485
gravity**(1/5)_1000 1 high    0.538715
                    2 medium  0.221689
                    3 low     0.167255
                    4 no      0.072340

In [24]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['San Antonio'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_San Antonio', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['United States']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\USA_San Antonio_2020.tif
get road networks from OSM
San Antonio done 3.5 mns
 
get urban greenspaces from OSM
San Antonio done
 
100m resolution grids extraction
San Antonio 17.87 mns

San Antonio
0.0 % fake entry points done 0.1  mns
14.0 % fake entry points done 1.37  mns
28.0 % fake entry points done 2.62  mns
42.0 % fake entry points done 3.83  mns
56.0 % fake entry points done 4.99  mns
70.0 % fake entry points done 6.32  mns
84.0 % fake entry points done 7.41  mns
98.0 % fake entry points done 8.54  mns
San Antonio 100 % fake entry points done 8.74  mns
San Antonio 100 % done 8.77  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
18.83 % 0.91 mns
37.65 % 1.83 mns
56.48 % 2.74 mns
75.3 % 3.65 mns
94.13 % 4.59 mns
100 % finding combinations done
San Antonio 342487 suitible combinations

obtain local graphs
San Antonio
0.0 % done 4.33 mns
18.83 % done 4.41 mns
37.65 % done 4.51 mn

City                          San Antonio
entrance_300        1 high       0.037851
                    2 medium     0.006623
                    3 low        0.030421
                    4 no         0.925105
entrance_600        1 high       0.041216
                    2 medium     0.037862
                    3 low        0.109631
                    4 no         0.811291
entrance_1000       1 high       0.057704
                    2 medium     0.101840
                    3 low        0.170026
                    4 no         0.670429
gravity**(1/2)_300  1 high       0.037632
                    2 medium     0.008617
                    3 low        0.033700
                    4 no         0.920052
gravity**(1/2)_600  1 high       0.039225
                    2 medium     0.044232
                    3 low        0.124978
                    4 no         0.791565
gravity**(1/2)_1000 1 high       0.053015
                    2 medium     0.115643
                    3 low        0.171326
                    4 no         0.660016
gravity**(1/3)_300  1 high       0.037615
                    2 medium     0.006970
                    3 low        0.029749
                    4 no         0.925666
gravity**(1/3)_600  1 high       0.039030
                    2 medium     0.038335
                    3 low        0.106639
                    4 no         0.815996
gravity**(1/3)_1000 1 high       0.051772
                    2 medium     0.101387
                    3 low        0.188340
                    4 no         0.658501
gravity**(1/5)_300  1 high       0.037615
                    2 medium     0.006409
                    3 low        0.028148
                    4 no         0.927828
gravity**(1/5)_600  1 high       0.039185
                    2 medium     0.036248
                    3 low        0.102508
                    4 no         0.822058
gravity**(1/5)_1000 1 high       0.052204
                    2 medium     0.097188
                    3 low        0.189433
                    4 no         0.661175

In [12]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Barcelona'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Barcelona', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Spain']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\ESP_Barcelona_2020.tif
get road networks from OSM
Barcelona done 4.53 mns
 
get urban greenspaces from OSM
Barcelona done
 
100m resolution grids extraction
Barcelona 3.53 mns

Barcelona
0.0 % fake entry points done 0.03  mns
12.8 % fake entry points done 1.83  mns
25.6 % fake entry points done 2.82  mns
38.4 % fake entry points done 3.8  mns
51.2 % fake entry points done 4.81  mns
63.9 % fake entry points done 5.69  mns
76.7 % fake entry points done 6.57  mns
89.5 % fake entry points done 7.51  mns
Barcelona 100 % fake entry points done 8.18  mns
Barcelona 100 % done 9.15  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
2.07 % 0.28 mns
4.14 % 0.77 mns
6.21 % 1.4 mns
8.28 % 2.12 mns
10.34 % 2.96 mns
12.41 % 4.04 mns
14.48 % 5.34 mns
16.55 % 6.97 mns
18.62 % 8.75 mns
20.69 % 10.8 mns
22.76 % 13.02 mns
24.83 % 15.27 mns
26.9 % 17.75 mns
28.96 % 20.22 mns
31.03 % 22.97 mns
33.1 % 25.86 mns
35.

KeyboardInterrupt: 

In [14]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Pune'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Pune', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['India']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\IND_Pune_2020.tif
get road networks from OSM
Pune done 1.1 mns
 
get urban greenspaces from OSM
Pune done
 
100m resolution grids extraction
Pune 0.51 mns

Pune
0.0 % fake entry points done 0.0  mns
27.5 % fake entry points done 0.52  mns
55.1 % fake entry points done 1.03  mns
82.6 % fake entry points done 1.53  mns
Pune 100 % fake entry points done 1.84  mns
Pune 100 % done 1.85  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
51.87 % 0.23 mns
100 % finding combinations done
Pune 187836 suitible combinations

obtain local graphs
Pune
0.0 % done 0.68 mns
51.87 % done 0.72 mns
100 % done 0.79 mns

Pune
0.0 % 0.1 mns
16.0 % 0.72 mns
31.99 % 1.35 mns
47.99 % 1.95 mns
63.98 % 2.6 mns
79.98 % 3.2 mns
95.97 % 3.74 mns
100 % done 4.24 mns

Pune
entrance 0.04 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 0.91 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 2.07 mns
grid  300
grid  600
grid  

City                              Pune
entrance_300        1 high    0.068722
                    2 medium  0.013219
                    3 low     0.065755
                    4 no      0.852304
entrance_600        1 high    0.093588
                    2 medium  0.080506
                    3 low     0.198401
                    4 no      0.627506
entrance_1000       1 high    0.182578
                    2 medium  0.165776
                    3 low     0.188004
                    4 no      0.463642
gravity**(1/2)_300  1 high    0.070950
                    2 medium  0.033854
                    3 low     0.110685
                    4 no      0.784511
gravity**(1/2)_600  1 high    0.133915
                    2 medium  0.137874
                    3 low     0.239236
                    4 no      0.488975
gravity**(1/2)_1000 1 high    0.285518
                    2 medium  0.198517
                    3 low     0.146148
                    4 no      0.369818
gravity**(1/3)_300  1 high    0.069031
                    2 medium  0.020307
                    3 low     0.077169
                    4 no      0.833493
gravity**(1/3)_600  1 high    0.103071
                    2 medium  0.095208
                    3 low     0.224763
                    4 no      0.576959
gravity**(1/3)_1000 1 high    0.226160
                    2 medium  0.193128
                    3 low     0.198771
                    4 no      0.381941
gravity**(1/5)_300  1 high    0.068664
                    2 medium  0.014970
                    3 low     0.066807
                    4 no      0.849560
gravity**(1/5)_600  1 high    0.093746
                    2 medium  0.083820
                    3 low     0.201432
                    4 no      0.621001
gravity**(1/5)_1000 1 high    0.192716
                    2 medium  0.181683
                    3 low     0.214631
                    4 no      0.410970

In [ ]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Chengdu'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Chengdu', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

In [30]:
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

Chengdu
0.0 % 0.09 mns
31.14 % 0.69 mns
62.27 % 1.26 mns
93.41 % 1.78 mns
100 % done 2.28 mns


In [31]:
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Chengdu', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

Chengdu
entrance 0.07 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 1.53 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 3.02 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 4.48 mns
grid  300
grid  600
grid  1000
Chengdu done 5.83 mns


City                           Chengdu
entrance_300        1 high    0.138580
                    2 medium  0.005865
                    3 low     0.032880
                    4 no      0.822675
entrance_600        1 high    0.144544
                    2 medium  0.043312
                    3 low     0.120757
                    4 no      0.691387
entrance_1000       1 high    0.189291
                    2 medium  0.107580
                    3 low     0.177834
                    4 no      0.525295
gravity**(1/2)_300  1 high    0.138453
                    2 medium  0.006443
                    3 low     0.030784
                    4 no      0.824320
gravity**(1/2)_600  1 high    0.140918
                    2 medium  0.041084
                    3 low     0.128061
                    4 no      0.689937
gravity**(1/2)_1000 1 high    0.173002
                    2 medium  0.115855
                    3 low     0.175252
                    4 no      0.535891
gravity**(1/3)_300  1 high    0.138457
                    2 medium  0.005403
                    3 low     0.028335
                    4 no      0.827805
gravity**(1/3)_600  1 high    0.140586
                    2 medium  0.037536
                    3 low     0.112941
                    4 no      0.708937
gravity**(1/3)_1000 1 high    0.168649
                    2 medium  0.108906
                    3 low     0.182619
                    4 no      0.539827
gravity**(1/5)_300  1 high    0.138449
                    2 medium  0.004637
                    3 low     0.028780
                    4 no      0.828134
gravity**(1/5)_600  1 high    0.141002
                    2 medium  0.037083
                    3 low     0.109948
                    4 no      0.711967
gravity**(1/5)_1000 1 high    0.171717
                    2 medium  0.103961
                    3 low     0.186864
                    4 no      0.537458

In [32]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Tallinn'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Tallinn', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Estonia']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\EST_Tallinn_2020.tif
get road networks from OSM
Tallinn done 1.63 mns
 
get urban greenspaces from OSM
Tallinn done
 
100m resolution grids extraction
Tallinn 4.1 mns

Tallinn
0.0 % fake entry points done 0.01  mns
24.0 % fake entry points done 0.87  mns
48.1 % fake entry points done 1.48  mns
72.1 % fake entry points done 2.03  mns
96.2 % fake entry points done 2.6  mns
Tallinn 100 % fake entry points done 2.68  mns
Tallinn 100 % done 2.84  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
5.84 % 0.26 mns
11.68 % 0.61 mns
17.52 % 1.05 mns
23.36 % 1.58 mns
29.21 % 2.21 mns
35.05 % 2.89 mns
40.89 % 3.67 mns
46.73 % 4.55 mns
52.57 % 5.48 mns
58.41 % 6.46 mns
64.25 % 7.49 mns
70.09 % 8.55 mns
75.93 % 9.67 mns
81.78 % 10.83 mns
87.62 % 12.03 mns
93.46 % 13.28 mns
99.3 % 14.57 mns
100 % finding combinations done
Tallinn 7304017 suitible combinations

obtain local graphs
Tallinn
0.0 % done 5.29 m

City                           Tallinn
entrance_300        1 high    0.174656
                    2 medium  0.114457
                    3 low     0.203796
                    4 no      0.507091
entrance_600        1 high    0.379235
                    2 medium  0.211795
                    3 low     0.215129
                    4 no      0.193840
entrance_1000       1 high    0.714548
                    2 medium  0.129432
                    3 low     0.081402
                    4 no      0.074618
gravity**(1/2)_300  1 high    0.156907
                    2 medium  0.069835
                    3 low     0.128339
                    4 no      0.644920
gravity**(1/2)_600  1 high    0.214322
                    2 medium  0.177949
                    3 low     0.226706
                    4 no      0.381024
gravity**(1/2)_1000 1 high    0.414301
                    2 medium  0.223237
                    3 low     0.226725
                    4 no      0.135737
gravity**(1/3)_300  1 high    0.156965
                    2 medium  0.065881
                    3 low     0.135043
                    4 no      0.642112
gravity**(1/3)_600  1 high    0.214954
                    2 medium  0.179348
                    3 low     0.234607
                    4 no      0.371092
gravity**(1/3)_1000 1 high    0.412241
                    2 medium  0.229470
                    3 low     0.217105
                    4 no      0.141184
gravity**(1/5)_300  1 high    0.158616
                    2 medium  0.072633
                    3 low     0.149761
                    4 no      0.618991
gravity**(1/5)_600  1 high    0.234864
                    2 medium  0.195248
                    3 low     0.239611
                    4 no      0.330277
gravity**(1/5)_1000 1 high    0.471809
                    2 medium  0.224720
                    3 low     0.183646
                    4 no      0.119825

In [12]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Kumamoto'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Kumamoto', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Japan']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\JPN_Kumamoto_2020.tif
get road networks from OSM
Kumamoto done 0.95 mns
 
get urban greenspaces from OSM
Kumamoto done
 
100m resolution grids extraction
Kumamoto 3.5 mns

Kumamoto
0.0 % fake entry points done 0.0  mns
70.9 % fake entry points done 0.22  mns
Kumamoto 100 % fake entry points done 0.31  mns
Kumamoto 100 % done 0.31  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
88.03 % 0.18 mns
100 % finding combinations done
Kumamoto 275020 suitible combinations

obtain local graphs
Kumamoto
0.0 % done 0.16 mns
88.03 % done 0.21 mns
100 % done 0.72 mns

Kumamoto
0.0 % 0.11 mns
16.89 % 0.76 mns
33.78 % 1.41 mns
50.67 % 1.96 mns
67.56 % 2.64 mns
84.45 % 3.35 mns
100 % done 3.86 mns

Kumamoto
entrance 0.04 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 0.59 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 1.32 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 1.88 mns
grid  300
grid  600


City                          Kumamoto
entrance_300        1 high    0.034576
                    2 medium  0.013120
                    3 low     0.045377
                    4 no      0.906927
entrance_600        1 high    0.044269
                    2 medium  0.059708
                    3 low     0.135094
                    4 no      0.760930
entrance_1000       1 high    0.109516
                    2 medium  0.118603
                    3 low     0.244323
                    4 no      0.527558
gravity**(1/2)_300  1 high    0.034138
                    2 medium  0.023226
                    3 low     0.082289
                    4 no      0.860347
gravity**(1/2)_600  1 high    0.037047
                    2 medium  0.114187
                    3 low     0.311985
                    4 no      0.536780
gravity**(1/2)_1000 1 high    0.189087
                    2 medium  0.280159
                    3 low     0.308805
                    4 no      0.221949
gravity**(1/3)_300  1 high    0.034082
                    2 medium  0.016260
                    3 low     0.052104
                    4 no      0.897554
gravity**(1/3)_600  1 high    0.036545
                    2 medium  0.071074
                    3 low     0.210392
                    4 no      0.681989
gravity**(1/3)_1000 1 high    0.065678
                    2 medium  0.226772
                    3 low     0.348547
                    4 no      0.359004
gravity**(1/5)_300  1 high    0.034258
                    2 medium  0.014204
                    3 low     0.044364
                    4 no      0.907174
gravity**(1/5)_600  1 high    0.038044
                    2 medium  0.058704
                    3 low     0.160592
                    4 no      0.742661
gravity**(1/5)_1000 1 high    0.072572
                    2 medium  0.153490
                    3 low     0.318187
                    4 no      0.455751

In [ ]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Bucharest'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Bucharest', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Romania']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\ROU_Bucharest_2020.tif
get road networks from OSM
Bucharest done 0.86 mns
 
get urban greenspaces from OSM
Bucharest done
 
100m resolution grids extraction
Bucharest 0.9 mns

Bucharest
0.0 % fake entry points done 0.0  mns
19.6 % fake entry points done 0.45  mns
39.1 % fake entry points done 0.82  mns
58.7 % fake entry points done 1.19  mns
78.3 % fake entry points done 1.56  mns
97.8 % fake entry points done 1.93  mns
Bucharest 100 % fake entry points done 1.97  mns
Bucharest 100 % done 2.01  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
9.92 % 0.5 mns
19.85 % 1.34 mns
29.77 % 2.35 mns
39.7 % 3.47 mns
49.62 % 4.71 mns
59.55 % 6.02 mns
69.47 % 7.36 mns
79.4 % 8.76 mns
89.32 % 10.2 mns
99.25 % 11.75 mns
100 % finding combinations done
Bucharest 8656809 suitible combinations

obtain local graphs
Bucharest
0.0 % done 2.52 mns
9.92 % done 2.69 mns
19.85 % done 3.52 mns
29.77 % done 4.52 mn

In [14]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Fukuoka'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
grid_scores = grid_score_summary (Dir_Routes, # Shortest routes by the Dijkstra algorithm, with gravity variant distance adj.
                                  cities_adj['City'], 
                                  population_grids, 
                                  ext = '_Fukuoka', # At multiple runs, the extention prevents the summarized file to be overwritten.
                                  save_path = 'D:/Dumps/GEE-WP Scores/Gravity_adj/',
                                  grid_size = 100) # Size of the grid in meters
grid_scores

['Japan']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\JPN_Fukuoka_2020.tif
get road networks from OSM



KeyboardInterrupt



In [3]:
def gee_worldpop_extract (city_file, iso, save_path = None):
    
    cities = city_file
    iso['name'] = np.where(iso['name'] == 'Macedonia','North Macedonia',iso['name'])
    
    # Get included city areas
    OSM_incl = [cities[cities['City'] == city]['OSM_area'].tolist()[0].rsplit(', ') for city in cities['City'].tolist()]

    # Get the city geoms
    obj = [city_geo(city).dissolve()['geometry'].tolist()[0] for city in OSM_incl]

    # Get the city countries
    obj_displ = [city_geo(city).dissolve()['display_name'].tolist()[0].rsplit(', ')[-1]for city in OSM_incl]
    print(obj_displ)
    obj_displ = np.where(pd.Series(obj_displ).str.contains("Ivoire"),"CIte dIvoire",obj_displ)

    # Get the country's iso-code
    iso_list = [iso[iso['name'] == ob]['alpha3'].tolist()[0] for ob in obj_displ]

    # Based on the iso-code return the worldpop 2020
    ee_worldpop = [ee.ImageCollection("WorldPop/GP/100m/pop")\
        .filter(ee.Filter.date('2020'))\
        .filter(ee.Filter.inList('country', [io])).first() for io in iso_list]

    # Clip the countries with the city geoms.
    clipped = [ee_worldpop[i].clip(shapely.geometry.mapping(obj[i])) for i in range(0,len(obj))]

    # Create path if non-existent
    if save_path == None:
        path = ''
    else:
        path = save_path
        if not os.path.exists(path):
                    os.makedirs(path)

    # Export as TIFF file.
    # Stored in form path + USA_Los Angeles_2020.tif
    filenames = [path+iso_list[i]+'_'+cities['City'][i]+'_2020.tif' for i in range(len(obj))]
    [geemap.ee_export_image(clipped[i], filename = filenames[i]) for i in range(0,len(obj))]
    return(filenames)
    sys.stdout.flush()
    
    # Block 2 Road networks
def road_network (cities, thresholds, undirected = False):
    print('get road networks from OSM')
    start_time = time.time()
    graphs = list()
    road_nodes = list()
    road_edges = list()
    road_conn = list()

    for i in enumerate(cities['OSM_area']):
        # Get graph, road nodes and edges
        road_node = pd.DataFrame()
        roads = pd.DataFrame()
        
        # For each included OSM_area get the roads
        for district in i[1].rsplit(', '):
            graph = ox.graph_from_place(district, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
            node, edge = ox.graph_to_gdfs(graph)
            road_node = pd.concat([road_node, node], axis = 0)
            roads = pd.concat([roads, edge], axis = 0)
        
        # Eliminate lists in the df which prevents drop of duplicate columns
        road_edge = pd.DataFrame([[c[0] if isinstance(c,list) else c for c in roads[col]]\
                              for col in roads]).transpose()
        road_edge.columns = roads.columns
        road_edge.index = roads.index
        road_edge = gpd.GeoDataFrame(road_edge, crs = 4326)
        
        # Return the unique nodes and edges of the (often) adjacent OSM_areas.
        road_node = road_node.drop_duplicates()
        road_edge = road_edge.drop_duplicates()
        
        # Road nodes format
        road_node = road_node.to_crs(4326)
        road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
        road_node['osmid_var'] = road_node.index
        road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)

        # format road edges
        road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
        road_edge = road_edge.reset_index()
        road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
        road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
        
        if undirected == True:
            # Apply one-directional to both for walking
            both = road_edge[road_edge['oneway'] == False]
            one = road_edge[road_edge['oneway'] == True]
            rev = pd.DataFrame()
            rev[['from','to']] = one[['to','from']]
            rev = pd.concat([rev,one.iloc[:,2:]],axis = 1)
            edge_bidir = pd.concat([both, one, rev])
            edge_bidir = edge_bidir.reset_index()
            edge_bidir['oneway'] = False
        else:
            edge_bidir = road_edge

        # Exclude highways and ramps on edges    
        edge_filter = edge_bidir[(edge_bidir['highway'].str.contains('motorway') | 
              (edge_bidir['highway'].str.contains('trunk') & 
               edge_bidir['maxspeed'].astype(str).str.contains(
                   '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
        road_edges.append(edge_filter)

        # Exclude isolated nodes
        fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
        newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
        road_nodes.append(newnodes)

        # Get only necessary road connections columns for network performance
        road_con = edge_filter[['osmid','key','length','geometry']]
        road_con = road_con.set_index('key')

        road_conn.append(road_con)

        # formatting to graph again.
        newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
        edge_filter = edge_filter.set_index(['from','to','keys'])
        edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]

        graph2 = ox.graph_from_gdfs(newnodes, edge_filter)

        graphs.append(graph2)
        print(cities['City'][i[0]].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    return({'graphs':graphs,'nodes':road_nodes,'edges':road_conn,'edges long':road_edges})
# Block 3 city greenspace
def urban_greenspace (cities, thresholds, one_UGS_buf = 25, min_UGS_size = 400):
    print('get urban greenspaces from OSM')
    parks_in_range = list()
    for i in enumerate(cities['OSM_area']):
        # Tags seen as Urban Greenspace (UGS) require the following:
        # 1. Tag represent an area
        # 2. The area is outdoor
        # 3. The area is (semi-)publically available
        # 4. The area is likely to contain trees, grass and/or greenery
        # 5. The area can reasonable be used for walking or recreational activities
        tags = {'landuse':['allotments','forest','greenfield','village_green'],\
                'leisure':['garden','fitness_station','nature_reserve','park','playground'],\
                'natural':'grassland'}
        gdf = ox.geometries_from_place(i[1].rsplit(', '),tags = tags,buffer_dist = np.max(thresholds))
        gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
        greenspace = gdf.reset_index()    
        warnings.filterwarnings("ignore")

        green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_UGS_buf).to_crs(4326))
        greenspace['geometry_w_buffer'] = green_buffer
        greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
        greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

        # This function group components in itself that overlap (with the buffer set of 25 metres)
        # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
        W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
        greenspace['components'] = W.component_labels
        parks = greenspace.dissolve('components')

        # Exclude parks below 0.04 ha.
        parks = parks[parks.to_crs(3043).area > min_UGS_size]
        print(cities['City'][i[0]], 'done')
        parks = parks.reset_index()
        parks['geometry_m'] = parks['geometry'].to_crs(3043)
        parks['park_area'] = parks['geometry_m'].area
        parks_in_range.append(parks)
    return(parks_in_range)
# Block 4 population grids extraction
def city_grids_format(city_grids, cities_area, road_nodes, UGS, grid_size = 100):
    start_time = time.time()
    grids = []
    print(str(grid_size) + 'm resolution grids extraction')
    for i in range(len(city_grids)):
        
        # Open the raster file
        with rasterio.open(city_grids[i]) as src:
            band= src.read() # the population values
            aff = src.transform # the raster bounds and size (affine)
        
        # Get the rowwise arrays, get a 2D dataframe
        grid = pd.DataFrame()
        for b in enumerate(band[0]):
            grid = pd.concat([grid, pd.Series(b[1],name=b[0])],axis=1)
        grid= grid.unstack().reset_index()
        
        # Unstack df to columns
        grid.columns = ['row','col','value']
        grid['minx'] = aff[2]+aff[0]*grid['col']
        grid['miny'] = aff[5]+aff[4]*grid['row']
        grid['maxx'] = aff[2]+aff[0]*grid['col']+aff[0]
        grid['maxy'] = aff[5]+aff[4]*grid['row']+aff[4]
        
        # Create polygon from affine bounds and row/col indices
        grid['geometry'] = [Polygon([(grid.minx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.maxy[i]),
                                   (grid.minx[i],grid.maxy[i])])\
                          for i in range(len(grid))]
        
        # Set the df as geo-df
        grid = gpd.GeoDataFrame(grid, crs = 4326) 

        # Get dissolvement_key for dissolvement. 
        grid['row3'] = np.floor(grid['row']/(grid_size/100)).astype(int)
        grid['col3'] = np.floor(grid['col']/(grid_size/100)).astype(int)
        grid['dissolve_key'] = grid['row3'].astype(str) +'-'+ grid['col3'].astype(str)
        
        # Define a city's OSM area as Polygon.
        geo_ls = gpd.GeoSeries(city_geo(cities_area[i].split(', ')).dissolve().geometry)
        
        # Intersect grids with the city boundary Polygon.
        insec = grid.intersection(geo_ls.tolist()[0])
        
        # Exclude grids outside the specified city boundaries
        insec = insec[insec.area > 0]
        
        # Join in other information.
        insec = gpd.GeoDataFrame(geometry = insec, crs = 4326).join(grid.loc[:, grid.columns != 'geometry'])
        
        # Dissolve into block by block grids
        popgrid = insec[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')
        
        # Get those grids populations and area. Only blocks with population and full blocks
        popgrid['population'] = round(insec.groupby('dissolve_key')['value'].sum()).astype(int)
        popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
        popgrid = popgrid[popgrid['population'] > 0]
        popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]

        # Get centroids and coords
        popgrid['centroid'] = popgrid['geometry'].centroid
        popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
        popgrid['grid_lon'] = popgrid['centroid_m'].x
        popgrid['grid_lat'] = popgrid['centroid_m'].y
        popgrid = popgrid.reset_index()

        minx = popgrid.bounds['minx']
        maxx = popgrid.bounds['maxx']
        miny = popgrid.bounds['miny']
        maxy = popgrid.bounds['maxy']

        # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters), coords error.
        # Therefore recreate the polygon.
        Poly = []
        for k in range(len(popgrid)):
            Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
        popgrid['geometry'] = Poly
        
        try:
            entry_index = [int(road_nodes[i]['geometry'].sindex.nearest(grid)[1])\
                                 for grid in popgrid['centroid']]
        except:
            entry_index = [int(road_nodes[i]['geometry'].sindex.nearest(grid)[1][0])\
                                 for grid in popgrid['centroid']]
            
        nearest_index = road_nodes[i].iloc[entry_index]
        popgrid['grid_osm'] = nearest_index.reset_index(drop = True)['osmid_var']
        popgrid['node_geom'] = nearest_index.reset_index(drop = True)['geometry']
        popgrid['node_geom_m'] = nearest_index.reset_index(drop = True)['geometry_m']
        popgrid['G-entry cost'] = popgrid['node_geom_m'].distance(popgrid['centroid_m'])
        
        UGS_all = UGS[i].dissolve().geometry[0]
        popgrid['in_out_UGS'] = popgrid.intersection(UGS_all).is_empty == False
        
        grids.append(popgrid)

        print(city_grids[i].rsplit('_')[3], round((time.time() - start_time)/60,2),'mns')
    return(grids)

In [4]:
# Block 5 park entry points
def UGS_fake_entry(UGS, road_nodes, graphs, cities, pop_grids,
                   thresholds, UGS_entry_buf = 25, walk_radius = 500, entry_point_merge = 0):
    
    start_time = time.time()
    ParkRoads = list()
    for j in range(len(cities)):
        ParkRoad = pd.DataFrame()
        mat = list()
        # For all
        print(cities[j].rsplit(',')[0])
        for i in range(len(UGS[j])):
            dist = road_nodes[j]['geometry'].to_crs(3043).distance(UGS[j]['geometry'].to_crs(
                3043)[i])
            buf_nodes = road_nodes[j][(dist < UGS_entry_buf) & (dist > 0)]
            mat.append(list(np.repeat(i, len(buf_nodes))))
            ParkRoad = pd.concat([ParkRoad, buf_nodes])
            if i % 100 == 0: print(round(i/len(UGS[j])*100,1),'% fake entry points done', 
                                  round((time.time() - start_time) / 60,2),' mns')
                
        print(cities[j].rsplit(',')[0],'100 % fake entry points done', round((time.time() - start_time) / 60,2),' mns')
        
        # Park no list conversion
        mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

        # Format
        ParkRoad['Park_No'] = mat_u
        ParkRoad = ParkRoad.reset_index()
        ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
        ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
        
        # Get the road nodes intersecting with the parks' buffer
        ParkRoad = pd.merge(ParkRoad, UGS[j][['geometry']], left_on = 'Park_No', right_index = True)

        # Get the walkable park size
        ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walk_radius).to_crs(4326).intersection(ParkRoad['geometry_y'])
        ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
        ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
        ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
        
        # Get size inflation factors for the gravity model
        ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
        ParkRoad['size_infl_sqr2'] = ParkRoad['size_infl_factor']**(1/2)
        ParkRoad['size_infl_sqr3'] = ParkRoad['size_infl_factor']**(1/3)
        ParkRoad['size_infl_sqr5'] = ParkRoad['size_infl_factor']**(1/5)
        ParkRoad['raw'] = 1
                
        # Merge fake UGS entry points if within X meters of each other for better system performance
        # Standard no merging
        ParkRoad = simplify_UGS_entry(ParkRoad, entry_point_merge = 0)
        ParkRoads.append(ParkRoad)
        
        print(cities[j].rsplit(',')[0],'100 % done', 
                                  round((time.time() - start_time) / 60,2),' mns')
        
    return(ParkRoads)
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)
def simplify_UGS_entry(fake_UGS_entry, entry_point_merge = 0):
    # Get buffer of nodes close to each other.
    # Get the buffer
    ParkComb = fake_UGS_entry
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(entry_point_merge)

    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels

    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']

    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    return(ParkComb)

In [5]:
def suitible_enhanced (UGS_entry, pop_grids, road_nodes, cities, thresholds):
    start_time = time.time()
    suits_all = []
    for j in range(len(cities)):
        print('get (Euclidean) suitible combinations')
        print('0.0 %', round((time.time() - start_time) / 60,2),'mns')
        UGSe = UGS_entry[j]
        entry_geoms = UGSe.geometry_m
        pop = pop_grids[j]
        road_node = road_nodes[j]

        suits = pd.DataFrame()
        cols = ['osmid','Park_No','walk_area','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5']
        for i in range(len(entry_geoms)):
            max_infl = np.max(UGSe[['raw','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5']], axis = 1)[i]
            suit_df = pop[pop.node_geom_m.distance(entry_geoms.iloc[i]) < (max_infl*np.max(thresholds))]
        
            suit_df['UGSe_osmid_m'] = entry_geoms.iloc[i]
            suit_df['Grid_No'] = suit_df.index
            suit_df = suit_df[['Grid_No','grid_osm','G-entry cost','in_out_UGS','node_geom_m','UGSe_osmid_m']].reset_index(drop = True)
            suit_df['Park_entry_No'] = UGSe.index[i]
            #suit_df = pd.merge(suit_df, UGSe[cols], left_on = 'Park_entry_No',right_index = True, how = 'left')
            suits = pd.concat([suits,suit_df])
            if (i+1) % 500 == 0: print(round((i+1) / len(entry_geoms)*100,2),'%',
                                       round((time.time() - start_time) / 60,2),'mns')
            
        suits = pd.merge(suits, UGSe[cols], left_on = 'Park_entry_No',right_index = True, how = 'left')
        suits = suits.reset_index(drop = True)
        suits = suits.rename(columns = {'osmid':'Parkroad_osmid','walk_area':'walk_area_m2'})
        suits['gridpark_no'] = suits['Grid_No'].astype(str)+'-'+suits['Park_No'].astype(str)
        suits['graph_key'] = suits['grid_osm'].astype(str)+'-'+suits['Parkroad_osmid'].astype(str)
        suits_all.append(suits)
        print('100 % finding combinations done')
        print(cities[j],len(suits),'suitible combinations')
    return(suits_all)

In [6]:
def obtaining_subgraphs(graphs, pop_grids, UGS_entry, nodes, cities, thresholds, time_sleep = 30):
    print('obtain local graphs')
    start_time = time.time()
    subgraphs_all = []
    suits_all = []
    for j in range(len(cities)):
        print(cities[j])
        Graph = graphs[j]
        pop = pop_grids[j]
        UGSe = UGS_entry[j].sort_values('osmid')
        road_node = nodes[j]
        node_geoms = road_node.geometry_m
        entry_geoms = UGSe.geometry_m
        osmid = UGSe['osmid']
        max_infl = np.max(UGSe[['raw','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5']], axis = 1)*(np.max(thresholds))

        dist = [node_geoms.distance(Point(i)) for i in entry_geoms]

        print('0.0 % done',round((time.time() - start_time) / 60,2),'mns')
        subgraphs = []
        UGSe_ids = []
        suits = pd.DataFrame()
        for i in range(len(entry_geoms)):      
            suit = road_node[['geometry_m']]
            suit['UGSe_osmid_m'] = entry_geoms.iloc[i]
            suit_df = dist[i]
            suit_in = suit_df[suit_df <= max_infl.iloc[i]]
            UGSe_ids.append(osmid.iloc[i])
            suit_in = pd.DataFrame(suit_in).join(node_geoms)
            suit_in['Parkroad_osmid'] = osmid.iloc[i]
            subgraphs.append(Graph.subgraph(suit_in.index))
            suits = pd.concat([suits, suit_in])

            if (i+1) % 500 == 0: 
                print(round((i+1) / len(entry_geoms)*100,2),'% done',
                                        round((time.time() - start_time) / 60,2),'mns')
                time.sleep(time_sleep)
        print('100 % done',round((time.time() - start_time) / 60,2),'mns')
        subgraphs_all.append(pd.Series(subgraphs, index = UGSe_ids))
        suits_all.append(suits)
    return({'graphs':subgraphs_all,'graph nodes':suits_all})

In [7]:
def distance_fast (Geo_1, Geo_2):
    return((abs(Geo_1.x - Geo_2.x)**2 + abs(Geo_1.y - Geo_2.y)**2).apply(math.sqrt))

In [8]:
def direct_routing (suitible_comb, graphs, edges, cities, chunk = 20000, time_sleep = 15):
    start_time = time.time()
    Routes = []
    Lines = []
    for j in enumerate(cities):
        print(j[1])
        
        suitible = suitible_comb[j[0]].sort_values('Parkroad_osmid').reset_index()
        grouped = suitible[suitible['in_out_UGS'] == False].groupby(['Parkroad_osmid'])['grid_osm'].apply(list)
        sets = grouped.apply(np.unique)

        Conn = edges[j[0]]
        SG = graphs[j[0]]
        
        SGr = SG.reset_index()
        SG = SGr.iloc[pd.Series(SGr['index'].drop_duplicates()).index].set_index('index')[0]

        num = int(np.ceil(chunk / sets.apply(len).mean()))
        length = int(np.ceil(len(suitible['Parkroad_osmid'].unique())/num))

        Routes_df = pd.DataFrame()
        Lines_df = pd.DataFrame()
        for l in range(length):
            comb = suitible[suitible['Parkroad_osmid'].isin(sets.index[l*num:l*num+num])]
            sets2 = sets[l*num:l*num+num]

            parknode = list(comb['Parkroad_osmid'])
            gridnode = list(comb['grid_osm'])
            subgraph = SG[sets2.index]

            ls = []
            ls2 = []
            ls3 = []
            lod = []
            lgk = []
            Routes
            for i in range(len(sets2)):
                path = nx.single_source_dijkstra(subgraph.iloc[i], sets2.index[i], weight = 'length')

                incl = np.isin(list(path[0].keys()),sets2.iloc[i])
                incl2 = np.isin(list(path[1].keys()),sets2.iloc[i])

                # route cost
                orig_c = list(np.repeat(sets2.index[i],sum(incl)))
                dest_c = list(np.array(list(path[0].keys()))[incl])
                cost = list(np.array(list(path[0].values()))[incl])

                ls = ls + orig_c
                ls2= ls2+ dest_c
                ls3= ls3+ cost

                # route steps
                orig_s = list(np.repeat(sets2.index[i],sum(incl2)))
                dest_s = list(np.array(list(path[1].keys()))[incl2])
                steps = list(np.array(list(path[1].values()),dtype=object)[incl2])

                fr = []
                to = []
                og = []
                de = []
                for j in enumerate(steps):
                    if len(j[1]) > 1:
                        fr.append(j[1][:-1])
                        to.append(j[1][1:])
                        og.append(list(np.repeat(orig_s[j[0]], len(j[1][:-1]))))
                        de.append(list(np.repeat(dest_s[j[0]], len(j[1][:-1]))))
                    else:
                        pass

                fr = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, fr) for i in b]
                to = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, to) for i in b]
                og = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, og) for i in b]
                de = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, de) for i in b]

                gk = [str(fr[k])+'-'+str(to[k]) for k in range(len(to))]
                gkr = [str(to[k])+'-'+str(fr[k]) for k in range(len(to))]
                od = [str(de[k])+'-'+str(og[k]) for k in range(len(og))]

                lgk.append(gk)
                lod.append(od)

            dist_df = pd.DataFrame({'UGSe_id':ls,'GrE_id':ls2,'route cost':ls3})
            dist_df['graph_key'] = dist_df['GrE_id'].astype(str)+'-'+dist_df['UGSe_id'].astype(str)

            routes = pd.merge(comb, dist_df, on = 'graph_key', how = 'left')
            routes['route cost'] = np.where(routes['in_out_UGS'],0,routes['route cost'])
            routes = routes[~routes['route cost'].isna()].reset_index(drop = True)

            routes['G-entry cost'] = np.where(routes['in_out_UGS'],0,routes['G-entry cost'])

            routes['raw_Tcost'] = routes['route cost']+routes['G-entry cost']
            routes['grav2_Tcost'] = routes['raw_Tcost'] / routes['size_infl_sqr2']
            routes['grav3_Tcost'] = routes['raw_Tcost'] / routes['size_infl_sqr3']
            routes['grav5_Tcost'] = routes['raw_Tcost'] / routes['size_infl_sqr5']

            lgk = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, lgk) for i in b]
            lod = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, lod) for i in b]
            
            linestr = pd.DataFrame({'route no':lod,'route step':lgk})
            
            linestr = pd.merge(linestr, Conn.geometry, left_on = 'route step', right_index = True, how = 'left')
            linestr = linestr[['route no','geometry']]
            linestr = gpd.GeoDataFrame(linestr[['route no','geometry']], crs = 4326)
            
            linestr = linestr.dissolve('route no')
            routes2 = pd.merge(routes, linestr, left_on = 'graph_key', right_index = True, how = 'left')
            
            Lines_df = pd.concat([Lines_df, linestr])
            Routes_df = pd.concat([Routes_df, routes2])
            
            print(round(l*num / len(sets)*100,2),'%', 
                  round((time.time() - start_time) / 60,2),'mns')
            time.sleep(time_sleep)
        Routes_df = Routes_df.sort_values('index')
        Routes_df = Routes_df.set_index('index')
        Routes_df = Routes_df.reset_index(drop = True)
        
        Routes_df = Routes_df[Routes_df.columns[~Routes_df.columns.isin(['UGSe_id', 'GrE_id','size_infl_sqr2',\
                                                                  'size_infl_sqr3', 'size_infl_sqr5'])]]
        
        print('100 % done',round((time.time() - start_time) / 60,2),'mns')
        
        Routes.append(Routes_df)
        Lines.append(Lines_df)
    return(Routes)

In [9]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)
def grid_score_summary (routes, cities, pop_grids, ext = '', grid_size = 100, save_path = 'C:/Dumps/GEE-WP Scores/Gravity/'):
    start_time = time.time()
    popg_acc = pd.DataFrame()
    grid_scores = list([])
    gridpark = list([])
    for n in range(len(cities)):    
        print(cities[n])

        # For the four distance decay variants regarding park size.
        l1 = list(['raw','grav2','grav3','grav5'])
        m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
        grid_score = list([])
        gridparks = list([])
        gridpark.append(gridparks)
        popgrid_access = pd.DataFrame()
        for i in range(len(l1)):
            # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
            var_best_routes = best_gridpark_comb (routes[n], l1[i], pop_grids[n])

            grdsc = pd.DataFrame()
            gridsc = pd.DataFrame()
            print(m1[i], round((time.time() - start_time) / 60,2), 'mns')

            # For each threshold given, calculate a score
            for k in range(len(thresholds)):
                
                t = thresholds[k]
                score = 'tr_'+ str(t)
                scores = determine_scores(var_best_routes, pop_grids[n], thresholds[k], l1[i], cities[n], 
                                          save_path, grid_size = 100)
                
                grdsc = pd.concat([grdsc, scores['score_w_route']], axis = 1)
                gridsc = pd.concat([gridsc, scores['grid_score']])
                                
                # Group according to the categories just created and sum the populations living in those grids
                popgacc = pd.DataFrame()
                popgacc[m1[i]+'_'+str(t)] = scores['score_w_route'].groupby(score+'_access')['population'].sum()
                popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   

                print('grid ',t)

            grid_score.append(grdsc)

            gridsc = gridsc.join(pop_grids[n]['geometry'])
            gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)

            if not os.path.exists(save_path+str(grid_size)+'m grids/Grid_geoms/'):
                os.makedirs(save_path+str(grid_size)+'m grids/Grid_geoms/')

            gridsc.to_file(save_path+str(grid_size)+'m grids/Grid_geoms/gridscore_'+ l1[i] + '_' + cities[n] + '.gpkg')

            # Detailed scores to files number of cities * ways to measure = number of files.
            # Different threshold-scores are in the same dataframe
            gridsc = gridsc.loc[:, gridsc.columns!='geometry']

            if not os.path.exists(save_path+str(grid_size)+'m grids/Grid_csv/'):
                os.makedirs(save_path+str(grid_size)+'m grids/Grid_csv/')

            gridsc.to_csv(save_path+str(grid_size)+'m grids/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
            gridparks.append(var_best_routes)

        grid_scores.append(grid_score)

        # For each city, divide the population access by group by the total to get its share.
        popgrid_access = popgrid_access / popgrid_access.sum()
        popgrid_access = pd.DataFrame(popgrid_access.unstack())
        popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)

        print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
    popg_acc.columns = cities
    popg_acc.to_csv(save_path+str(grid_size)+'m grids/popgrid_access.csv')
    return(popg_acc)    


In [10]:
def best_gridpark_comb (routes, var_abbr, pop_grid):
    str1 = 'gridpark_' + var_abbr
    locals()[str1] = routes.iloc[routes.groupby('gridpark_no')[(str(var_abbr) +'_Tcost')].idxmin()]  

    # Get grid information
    locals()[str1] = pd.merge(locals()[str1], pop_grid[['population','geometry']],
                            left_on = 'Grid_No', right_index = True, how = 'outer')
    locals()[str1] = locals()[str1].reset_index()

    # formatting
    locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
    locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
    locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
    locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
    return(locals()[str1])

In [11]:
def determine_scores(var_df, pop_grid, thresholds, var_abbr, city, save_path, grid_size = 100):
    t = thresholds
    str2 = str(t)
    score = 'tr_'+ str2

    #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
    thold = var_df[var_df[var_abbr + '_Tcost'] <= t]
    thold[score] = t - thold[var_abbr + '_Tcost']
    thold['pop' + score] = thold[score] * thold['population']
    thold['walk_area_ha' + str2] = var_df['walk_area_m2'] /10000
    thold['walkha_person' + str2] = thold['population'] / thold['walk_area_ha' + str2]

    # Join the gridpark information from before.
    var_df = var_df.join(thold[[score,'pop' + score,'walk_area_ha' + str2, 'walkha_person' + str2]])
    # get the grid_scores
    gs = pd.DataFrame()
    gs[[score,'pop_' + score,'walkha_' + str2]] = var_df.groupby(
            'Grid_No')[score,'pop' + score, 'walk_area_ha' + str2].sum()

    gs['walkha_person_' + score] = var_df.groupby('Grid_No')['walkha_person' + str2].mean()

    trstr = var_df[var_df[score] > 0]
    gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()

    # Add the routes as a dissolved line_geom
    gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                  geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

    # Add parks which grids have access to with its closest access point
    gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
    ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
                
    # determine the thresholds category-score. 
    # High >= threshold (perfect score to one park), medium is above half perfect, 
    # low is below this and no is no access to a park for a certain grid within the threshold given
    gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
    gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
          ['1 high','2 medium','3 low','4 no'])
    gs = gs.join(pop_grid['population'], how = 'outer')
            
    gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            
    if not os.path.exists(save_path+str(grid_size)+'m grids/Grid_lines/'):
        os.makedirs(save_path+str(grid_size)+'m grids/Grid_lines/')
                
    gs.to_file(save_path+str(grid_size)+'m grids/Grid_lines/gridscore_'+ var_abbr + '_' + str2 + '_' + city + '.gpkg')
            
    gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]

    return({'grid_score':gsc,'score_w_route':gs})